In [ ]:
import mysql.connector
from mysql.connector import Error

def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Mysql@1234"
        )
        print("MySQL Database connection successful")
    except Error as e:
        print(f"Error: '{e}'")
    
    return connection

def create_database(connection, db_name):
    cursor = connection.cursor()
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        print(f"Database '{db_name}' created successfully")
    except Error as e:
        print(f"Error: '{e}'")

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"Error: '{e}'")

def create_tables(connection):
    # Use the IQ test database
    cursor = connection.cursor()
    cursor.execute("USE iq_test_db")
    
    # Create Users table
    create_users_table = """
    CREATE TABLE IF NOT EXISTS users (
        user_id INT AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(50) NOT NULL UNIQUE,
        password VARCHAR(255) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        is_admin BOOLEAN DEFAULT FALSE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    
    # Create Question Categories table
    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        description TEXT
    );
    """
    
    # Create Questions table
    create_questions_table = """
    CREATE TABLE IF NOT EXISTS questions (
        question_id INT AUTO_INCREMENT PRIMARY KEY,
        category_id INT,
        section_id INT,  # To identify which section (1, 2, or 3) the question belongs to
        question_type ENUM('text_only', 'image_only', 'text_and_image') NOT NULL,
        question_text TEXT,
        image_path VARCHAR(255),  # Path to image file if question includes an image
        difficulty INT CHECK (difficulty BETWEEN 1 AND 10),
        num_options INT NOT NULL DEFAULT 4,  # Number of options (4 for A-D, 6 for A-F)
        time_limit INT,  # Time limit in seconds
        created_by INT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (category_id) REFERENCES categories(category_id),
        FOREIGN KEY (created_by) REFERENCES users(user_id)
    );
    """
    
    # Create Answer Options table
    create_options_table = """
    CREATE TABLE IF NOT EXISTS answer_options (
        option_id INT AUTO_INCREMENT PRIMARY KEY,
        question_id INT,
        option_text TEXT NOT NULL,
        is_correct BOOLEAN DEFAULT FALSE,
        FOREIGN KEY (question_id) REFERENCES questions(question_id) ON DELETE CASCADE
    );
    """
    
    # Create Tests table
    create_tests_table = """
    CREATE TABLE IF NOT EXISTS tests (
        test_id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        description TEXT,
        time_limit INT DEFAULT 60,  # Total time limit in minutes (default 60)
        section1_questions INT DEFAULT 10,  # Number of questions to randomly select from section 1
        section2_questions INT DEFAULT 10,  # Number of questions to randomly select from section 2
        section3_questions INT DEFAULT 10,  # Number of questions to randomly select from section 3
        passing_score INT,
        created_by INT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (created_by) REFERENCES users(user_id)
    );
    """
    
    # Create Test Sessions table (for proctoring)
    create_test_sessions_table = """
    CREATE TABLE IF NOT EXISTS test_sessions (
        session_id INT AUTO_INCREMENT PRIMARY KEY,
        test_id INT,
        user_id INT,
        start_time TIMESTAMP,
        end_time TIMESTAMP,
        status ENUM('in_progress', 'completed', 'aborted') DEFAULT 'in_progress',
        ip_address VARCHAR(45),
        browser_info TEXT,
        FOREIGN KEY (test_id) REFERENCES tests(test_id),
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    );
    """
    
    # Create User Answers table
    create_user_answers_table = """
    CREATE TABLE IF NOT EXISTS user_answers (
        answer_id INT AUTO_INCREMENT PRIMARY KEY,
        session_id INT,
        question_id INT,
        selected_option_id INT,
        answer_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        time_spent INT,  # Time spent on this question in seconds
        FOREIGN KEY (session_id) REFERENCES test_sessions(session_id) ON DELETE CASCADE,
        FOREIGN KEY (question_id) REFERENCES questions(question_id),
        FOREIGN KEY (selected_option_id) REFERENCES answer_options(option_id)
    );
    """
    
    # Create Proctoring Events table
    create_proctoring_events_table = """
    CREATE TABLE IF NOT EXISTS proctoring_events (
        event_id INT AUTO_INCREMENT PRIMARY KEY,
        session_id INT,
        event_type ENUM('tab_switch', 'focus_loss', 'copy_attempt', 'multiple_faces', 'no_face', 'suspicious_movement', 'summary'),
        event_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        event_data TEXT,  # Can store JSON or details about the event
        FOREIGN KEY (session_id) REFERENCES test_sessions(session_id) ON DELETE CASCADE
    );
    """
    
    # Execute all table creation queries
    execute_query(connection, create_users_table)
    execute_query(connection, create_categories_table)
    execute_query(connection, create_questions_table)
    execute_query(connection, create_options_table)
    execute_query(connection, create_tests_table)
    execute_query(connection, create_test_sessions_table)
    execute_query(connection, create_user_answers_table)
    execute_query(connection, create_proctoring_events_table)

def main():
    # Create a connection to MySQL
    connection = create_connection()
    
    if connection is not None:
        # Create database
        create_database(connection, "iq_test_db")
        
        # Create all tables
        create_tables(connection)
        
        # Close connection
        connection.close()
        print("MySQL connection is closed")
        print("Database setup complete! You can now run the admin application.")
    else:
        print("Failed to connect to MySQL")

if __name__ == "__main__":
    main()

MySQL Database connection successful
Database 'iq_test_db' created successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
MySQL connection is closed
Database setup complete! You can now run the admin application.
